In [ ]:
# this was just a lot of ideas, and i wanted a place to write it down. A journal of sorts i guess.

In [ ]:
# okay, the first one is a quick function / utility. 
# this if made super easy (import sketch as sk; sk.Init(sk.openAI('1234'))) 
# then you can do df.groupby('state').apply(sk.prompt("Colors for NBA fans in Zion: [0xF12C2FE, 0x012345]\nColors for NBA fans in {{ state }}:"))
# ## sk.prompts have a "clean" built in, that ensures structure is right. In this example, it infers that it should be a list of colors (in hex)
# sk.__get_total_number_of_users(..)

# not all prompts work. There is a "fee" on openAI per fill-in, so there's a billing thing there
# can offer "Reduced billing" (if it's known that most prompts that come in can be run as batches)
# eg. those that are run behind pandas like this, are more likely to have a "parallelizable" "apply" structure.
# those (i think) are cheaper to run - based on the "run inference as fast as possible" libraries which have "batch sized based scaling" aparently (linear in that is 'almost free', it's a trade-off dimension?)
# so, if you have a prompt that is "batchable" (like this one) you can get a discount on the "fill-in" cost.

# so, the next one is a "fitbit" dashboard.
# this is a bit more complicated, but it's a good example of how to use the "sketch" library to build a dashboard. -> edit / append to, a dashboard

# could lean into the idea of "sketch" (draw on top of, add a sketch-pad on top, that's a 'digital sandbox' that you can move and put shapes in?? lol, im stretching here, doesn't work really)

In [ ]:
from sketch.examples.prompt_machine import *

import pandas as pd

In [ ]:
# Data search code

# load any data you want (including using st helper functions)
state_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')[1]
state_data.columns = state_data.columns.to_flat_index()
state_names = state_data[state_data.columns[0]]

In [ ]:
state_names.head()

In [ ]:
result = state_names.apply(
    lambda x: GPT3Prompt(
        "get_colors_for_state",
        "Colors for NBA fans in {{ state }} (as a list of hex tuple, eg. (0xF1F1F1, ...)): (0x"
        )(state=x)
)

In [ ]:
# Let's color some maps with the colors we got from GPT-3 (from a language)
# oh no, choices error.

In [ ]:
#  convert  `F1F1F1, 0xF1F1F1, 0xF1F1F1, 0xF1F1F1, 0xF1F1F1)` -> [F1F1F1, F1F1F1, F1F1F1, F1F1F1, F1F1F1]
# 1                   or        `BD1021, 0x002244, 0xFDB927)` -> [BD1021, 002244, FDB927]`

def clean_colors(x):
    x = x.replace('(', '').replace(')', '').replace(' ', '').replace('0x', '')
    return [y for y in x.split(',') if len(y) == 6]

In [ ]:
colors_as_list = result.apply(clean_colors)

In [ ]:
import altair as alt

state_names # state names is a a list of state names like "Alabama"
colors_as_list # is a pandas series that has colors in a list format ["F1F2F3", "C1C2C3"]

alt.Chart(state_names).mark_geoshape().encode(
    color=alt.Color('state:N', scale=alt.Scale(domain=state_names, range=colors_as_list))
).project(
    type='albersUsa'
)